<a href="https://colab.research.google.com/github/ENGCSS/Trabalho_Analise_espacial_de_redes/blob/main/Tb_Redes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>